In [2]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import  LabelEncoder

<h3>Загрузка и предобработка датасета</h3>

In [ ]:
#загрузка датасета
df = pd.read_csv('coords_data_numerated.csv')
df = df.drop(columns=['Unnamed: 0', 'GSR'])
df.head()

In [ ]:
#Определение выходного признака и входных признаков
y = df['BreathingType']
X = df.drop(columns=['BreathingType'])
X.tail()

In [ ]:
#переименование столбцов
dict_renames = {
    'FirstMarkerXCoord' : 'FMX',
    'FirstMarkerYCoord' : 'FMY',
    'FirstMarkerZCoord' : 'FMZ',
    'SecondMarkerXCoord' :  'SMX',
    'SecondMarkerYCoord' : 'SMY',
    'SecondMarkerZCoord' : 'SMZ',
    'ThirdMarkerXCoord' : 'TMX',
    'ThirdMarkerYCoord' : 'TMY',
    'ThirdMarkerZCoord' : 'TMZ'
}
X = X.rename(columns=dict_renames)
X.head()

<h3>Извлечение признаков с помощью библиотеки TSFresh</h3>

In [6]:
import tsfresh

In [19]:
settings = tsfresh.feature_extraction.settings.EfficientFCParameters()

In [20]:
extracted_features = tsfresh.extract_features(X, column_sort='TimeStamp_sec', column_id='id', default_fc_parameters=settings)

Feature Extraction: 100%|██████████| 10/10 [04:31<00:00, 27.12s/it]


In [21]:
extracted_features.head()

,FMX__variance_larger_than_standard_deviation,FMX__has_duplicate_max,FMX__has_duplicate_min,FMX__has_duplicate,FMX__sum_values,FMX__abs_energy,FMX__mean_abs_change,FMX__mean_change,FMX__mean_second_derivative_central,FMX__median,...,TMZ__fourier_entropy__bins_5,TMZ__fourier_entropy__bins_10,TMZ__fourier_entropy__bins_100,TMZ__permutation_entropy__dimension_3__tau_1,TMZ__permutation_entropy__dimension_4__tau_1,TMZ__permutation_entropy__dimension_5__tau_1,TMZ__permutation_entropy__dimension_6__tau_1,TMZ__permutation_entropy__dimension_7__tau_1,TMZ__query_similarity_count__query_None__threshold_0.0,TMZ__mean_n_absolute_max__number_of_maxima_7
1,0.0,0.0,1.0,1.0,594.610,392.525460,0.000393,-0.000024,1.668521e-06,0.661,...,0.170467,0.249958,0.620602,1.371588,2.146117,2.940803,3.707134,4.381382,NaN,4.381000
2,0.0,1.0,1.0,1.0,600.119,399.859511,0.000433,-0.000044,0.000000e+00,0.667,...,0.228465,0.382629,0.886176,1.317787,2.063640,2.838046,3.611423,4.321921,NaN,4.375286
3,0.0,1.0,1.0,1.0,580.441,374.072781,0.000484,-0.000029,-5.561735e-07,0.643,...,0.235155,0.339942,0.691312,1.409429,2.237123,3.090561,3.928044,4.640799,NaN,4.386000
4,0.0,1.0,1.0,1.0,762.231,645.009181,0.000381,-0.000026,-1.112347e-06,0.848,...,0.090729,0.090729,0.523421,1.375399,2.188237,3.025499,3.862329,4.636355,NaN,4.248714
5,0.0,1.0,0.0,1.0,589.421,385.611861,0.000332,0.000017,0.000000e+00,0.655,...,0.346333,0.465406,0.996480,1.321429,2.013405,2.718388,3.407454,4.044901,NaN,4.188000


In [ ]:
y = pd.Series(y['BreathingType'])

In [23]:
y.head()

id
1    0
2    1
3    2
4    0
5    1
Name: BreathingType, dtype: int64

In [24]:
from tsfresh import select_features
from tsfresh.utilities.dataframe_functions import impute

impute(extracted_features)
features_filtered = select_features(extracted_features, y, multiclass=True)

C:\Users\admin\PycharmProjects\TimeSeriesLibraries\venv\lib\site-packages\tsfresh\utilities\dataframe_functions.py:198: RuntimeWarning: The columns ['FMX__query_similarity_count__query_None__threshold_0.0'
 'FMY__query_similarity_count__query_None__threshold_0.0'
 'FMZ__query_similarity_count__query_None__threshold_0.0'
 'SMX__query_similarity_count__query_None__threshold_0.0'
 'SMY__friedrich_coefficients__coeff_0__m_3__r_30'
 'SMY__friedrich_coefficients__coeff_1__m_3__r_30'
 'SMY__friedrich_coefficients__coeff_2__m_3__r_30'
 'SMY__friedrich_coefficients__coeff_3__m_3__r_30'
 'SMY__max_langevin_fixed_point__m_3__r_30'
 'SMY__query_similarity_count__query_None__threshold_0.0'
 'SMZ__query_similarity_count__query_None__threshold_0.0'
 'TMX__query_similarity_count__query_None__threshold_0.0'
 'TMY__query_similarity_count__query_None__threshold_0.0'
 'TMZ__query_similarity_count__query_None__threshold_0.0'] did not have any finite values. Filling with zeros.
  warnings.warn(


In [25]:
features_filtered.head()

,"FMY__agg_autocorrelation__f_agg_""var""__maxlag_40","FMY__agg_linear_trend__attr_""intercept""__chunk_len_50__f_agg_""var""",FMY__partial_autocorrelation__lag_3,FMY__fourier_entropy__bins_10,"FMY__agg_linear_trend__attr_""stderr""__chunk_len_10__f_agg_""mean""","FMY__linear_trend__attr_""stderr""","FMY__agg_linear_trend__attr_""stderr""__chunk_len_5__f_agg_""mean""","FMY__agg_linear_trend__attr_""stderr""__chunk_len_5__f_agg_""max""","FMY__agg_linear_trend__attr_""stderr""__chunk_len_5__f_agg_""min""","FMY__agg_linear_trend__attr_""stderr""__chunk_len_10__f_agg_""max""",...,"TMX__agg_linear_trend__attr_""intercept""__chunk_len_5__f_agg_""var""","TMX__agg_linear_trend__attr_""stderr""__chunk_len_10__f_agg_""max""","TMX__agg_linear_trend__attr_""stderr""__chunk_len_10__f_agg_""var""","TMX__change_quantiles__f_agg_""mean""__isabs_True__qh_1.0__ql_0.8","TMX__agg_linear_trend__attr_""stderr""__chunk_len_5__f_agg_""var""","TMX__change_quantiles__f_agg_""var""__isabs_False__qh_1.0__ql_0.8","TMX__change_quantiles__f_agg_""var""__isabs_True__qh_1.0__ql_0.8",TMX__ar_coefficient__coeff_3__k_10,"TMX__agg_linear_trend__attr_""stderr""__chunk_len_50__f_agg_""var""",TMX__ar_coefficient__coeff_1__k_10
1,0.203342,0.000013,-0.222898,0.136002,0.000012,4.290971e-07,0.000005,0.000005,0.000005,0.000013,...,1.293342e-06,0.000045,2.114180e-08,0.000635,2.229450e-09,9.175544e-07,5.300205e-07,-0.069700,1.498184e-06,1.075671
2,0.368654,0.000055,-0.325654,0.136002,0.000031,1.024434e-06,0.000011,0.000011,0.000011,0.000032,...,3.231085e-07,0.000028,8.509713e-09,0.000358,7.471823e-10,3.678488e-07,2.400601e-07,-0.005454,7.830371e-07,0.936582
3,0.368037,0.000052,-0.381678,0.136002,0.000035,1.141785e-06,0.000013,0.000013,0.000013,0.000034,...,1.314077e-06,0.000055,2.155637e-08,0.000715,2.006410e-09,1.060953e-06,5.501077e-07,0.016064,2.354984e-06,0.937611
4,0.029265,0.000003,-0.126742,0.280179,0.000009,2.894390e-07,0.000003,0.000003,0.000003,0.000009,...,7.638346e-07,0.000036,1.435572e-08,0.000398,1.644779e-09,4.077468e-07,2.497918e-07,-0.130670,7.360300e-07,1.090171
5,0.254175,0.000007,-0.265776,0.170467,0.000010,3.465600e-07,0.000004,0.000004,0.000004,0.000011,...,2.256864e-07,0.000014,2.989264e-09,0.000254,3.413762e-10,2.613493e-07,1.969553e-07,0.093239,1.301934e-07,0.831451


In [26]:
features_filtered.to_csv('features_filtered.csv')

In [30]:
new_features = features_filtered.reset_index()

<h3>Добавление новых признаков в датасет</h3>

In [31]:
#загрузка датасета
data = pd.read_csv('breathes.csv')
data = data.drop(columns=['Unnamed: 0'])
data.head()

,ID,Date_Of_Birth,Sex,Had_Covid,Begin_Of_Covid,End_Of_Covid,Lung_Damage,Damage_Percent,Breathing_Type,Frequency,...,Sin_Period23,Sin_Amp23/Sin_Amp13,Sin_Amp12/Sin_Amp13,Sin_Amp12/Sin_Amp23,Sin_Omega23/Sin_Omega13,Sin_Omega12/Sin_Omega13,Sin_Omega12/Sin_Omega23,(Sin_Offset23/Sin_Offset13)^2,(Sin_Offset12/Sin_Offset13)^2,(Sin_Offset12/Sin_Offset23)^2
0,1,04.07.2003,M,No,00.00.0000,00.00.0000,No,-0.207005,грудное,-0.144237,...,-0.178712,0.002273,-0.001832,-0.032392,-0.156378,-0.155622,-0.156309,-0.088206,-0.063808,-0.059907
1,1,04.07.2003,M,No,00.00.0000,00.00.0000,No,-0.207005,грудное,-0.144237,...,-0.183415,0.002486,-0.001808,-0.029759,-0.151883,-0.154821,-0.159698,-0.088211,-0.063809,-0.059906
2,1,04.07.2003,M,No,00.00.0000,00.00.0000,No,-0.207005,грудное,-0.144237,...,-0.177844,0.002607,-0.001792,-0.030606,-0.157144,-0.156177,-0.155971,-0.088215,-0.063811,-0.059905
3,1,04.07.2003,M,No,00.00.0000,00.00.0000,No,-0.207005,грудное,-0.144237,...,-0.182131,0.002219,-0.002349,-0.030553,-0.152623,-0.155988,-0.160060,-0.088219,-0.063815,-0.059905
4,1,04.07.2003,M,No,00.00.0000,00.00.0000,No,-0.207005,грудное,-0.144237,...,-0.191648,0.003473,-0.001519,-0.031048,-0.165326,-0.156451,-0.137226,-0.088204,-0.063811,-0.059908


In [ ]:
#добавление новых признаков в датасет
i = 0
new_features_cols = new_features.columns
for id in range(1,87):
    for col in new_features_cols:
        data.loc[(data['Breathing_Type'] == 'грудное') & (data['ID'] == id), col]= new_features.loc[i][col]
    i += 1
    for col in new_features_cols:
        data.loc[(data['Breathing_Type'] == 'брюшное') & (data['ID'] == id), col] = new_features.loc[i][col]
    i += 1
    for col in new_features_cols:
        data.loc[(data['Breathing_Type'] == 'смешанное') & (data['ID'] == id), col] = new_features.loc[i][col]
    i += 1

In [ ]:
data.head(15)

<h3>Подготовка датасета для тестирования</h3>

In [35]:
#отбрасывание ненужных столбцов
categorical_cols = ['Sex', 'Had_Covid', 'Lung_Damage', 'DominatorFreq']
drop_columns = ['Date_Of_Birth', 'Begin_Of_Covid', 'End_Of_Covid'] + categorical_cols
df = data.drop(columns=drop_columns)
#целевой признак
y = df['Breathing_Type']
#определение численных признаков
non_numerical_cols = ['ID', 'Breathing_Type']
X = df.drop(columns=non_numerical_cols)
#трансформация категориальных признаков
le = LabelEncoder()
y = le.fit_transform(y)

In [36]:
#разделение на обучающую(70%) и тестовую (30%) выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [37]:
from sklearn.metrics import recall_score, accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import classification_report
#расчет метрик по тесовой выборке
def calculate_metrics(y_test, y_pred):
    accuracy = accuracy_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred, average='weighted')
    precision = precision_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    print("Accuracy: ", accuracy)
    print("Recall: ", recall)
    print("Precision: ", precision)
    print("F1-score: ", f1)
    report = classification_report(y_test, y_pred)
    print(report)

#расчет знамости признаков
def calc_feature_importances(model, X_df):
    importance = model.feature_importances_
    feature_importance = pd.DataFrame({'Feature': X_df.columns, 'Importance': importance})
    feature_importance = feature_importance.sort_values('Importance', ascending=False)
    return feature_importance

<h2>Тестирование новых признаков</h2>

<h3>Generic Univariate Select (Mutual Info)</h3>

In [69]:
from sklearn.feature_selection import GenericUnivariateSelect, mutual_info_classif

selector = GenericUnivariateSelect(mutual_info_classif, mode='k_best', param=50)

In [70]:
X_new = selector.fit_transform(X, y)

In [71]:
selector.get_feature_names_out()
selector.get_support()

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False,  True,  True,
        True, False, False,  True,  True,  True, False,  True, False,
        True, False,  True, False, False, False, False, False, False,
        True, False,  True,  True, False, False,  True, False,  True,
        True,  True,  True, False, False,  True, False, False, False,
       False,  True,  True,  True, False, False, False, False, False,
       False, False,  True,  True, False, False,  True, False, False,
       False, False, False,  True,  True, False,  True,  True,  True,
       False, False, False, False, False, False, False, False, False,
       False, False,  True, False, False,  True, False, False, False,
       False, False, False, False,  True, False, False, False, False,
        True,  True,

In [72]:
importance = selector.scores_
feature_importance = pd.DataFrame({'Feature': X.columns, 'Scores': importance, 'Support': selector.get_support()})
feature_importance = feature_importance.sort_values('Scores', ascending=False)
feature_importance

,Feature,Scores,Support
47,"FMY__agg_linear_trend__attr_""intercept""__chunk...",1.096773,True
243,"FMZ__fft_coefficient__attr_""real""__coeff_37",1.092985,True
210,"FMX__agg_linear_trend__attr_""stderr""__chunk_le...",1.091748,True
39,"FMY__linear_trend__attr_""stderr""",1.091177,True
121,FMY__spkt_welch_density__coeff_2,1.090960,True
...,...,...,...
18,Sin_Amp23,0.013608,False
0,Damage_Percent,0.010196,False
3,DominatorFreqPower,0.008057,False
20,Sin_Phase23,0.000000,False


In [59]:
feature_importance.to_excel('mutual_features.xlsx')

<h3>Generic Univariate Select (F-Test)</h3>

In [79]:
from sklearn.feature_selection import f_classif

selector = GenericUnivariateSelect(f_classif, mode='k_best', param=50)

In [80]:
X_new = selector.fit_transform(X, y)

In [81]:
importance = selector.scores_
feature_importance = pd.DataFrame({'Feature': X.columns, 'Scores': importance})
feature_importance = feature_importance.sort_values('Scores', ascending=False)
feature_importance

,Feature,Scores
34,"FMY__agg_autocorrelation__f_agg_""var""__maxlag_40",206.868797
36,FMY__partial_autocorrelation__lag_3,186.130765
37,FMY__fourier_entropy__bins_10,166.267675
46,FMY__fourier_entropy__bins_100,163.495489
41,"FMY__agg_linear_trend__attr_""stderr""__chunk_le...",147.095832
...,...,...
23,Sin_Period23,0.221363
0,Damage_Percent,0.164475
30,(Sin_Offset23/Sin_Offset13)^2,0.139079
16,Sin_Offset13,0.136538


In [55]:
feature_importance.to_excel('ftest_features.xlsx')

<h3>RandomForest</h3>

In [82]:
from sklearn.ensemble import RandomForestClassifier

model_rf = RandomForestClassifier()

In [83]:
#разделение на обучающую(70%) и тестовую (30%) выборки
X_train, X_test, y_train, y_test = train_test_split(X_new, y, test_size=0.3, random_state=42)

In [84]:
details = model_rf.fit(X_train, y_train)
y_pred = model_rf.predict(X_test)
calculate_metrics(y_test, y_pred)

Accuracy:  0.9905437352245863
Recall:  0.9905437352245863
Precision:  0.9906042795368738
F1-score:  0.9905428945182585
              precision    recall  f1-score   support

           0       0.99      0.98      0.99       149
           1       1.00      1.00      1.00       123
           2       0.98      0.99      0.99       151

    accuracy                           0.99       423
   macro avg       0.99      0.99      0.99       423
weighted avg       0.99      0.99      0.99       423



In [85]:
model_rf.feature_importances_

array([0.0402665 , 0.04113919, 0.04700932, 0.03105758, 0.02463717,
       0.0014214 , 0.02064368, 0.01051424, 0.01010471, 0.01903452,
       0.02179238, 0.00366105, 0.03282354, 0.02536244, 0.02341634,
       0.0154767 , 0.01847713, 0.02478652, 0.00143219, 0.00104913,
       0.02456719, 0.00043256, 0.02817114, 0.01849144, 0.02723226,
       0.02223125, 0.01831052, 0.00045465, 0.03682821, 0.01178156,
       0.00031401, 0.00146025, 0.02464305, 0.01701067, 0.03250895,
       0.02028943, 0.03510181, 0.0244807 , 0.01947358, 0.00059465,
       0.01618317, 0.0187426 , 0.01645794, 0.01767952, 0.01803394,
       0.01712506, 0.03108176, 0.02084425, 0.02186015, 0.04350801])

In [86]:
importance = model_rf.feature_importances_
feature_importance = pd.DataFrame({'Feature': selector.get_feature_names_out(), 'Importance': importance})
feature_importance = feature_importance.sort_values('Importance', ascending=False)
feature_importance

,Feature,Importance
2,FMY__partial_autocorrelation__lag_3,0.047009
49,"TMX__fft_coefficient__attr_""abs""__coeff_33",0.043508
1,"FMY__agg_linear_trend__attr_""intercept""__chunk...",0.041139
0,"FMY__agg_autocorrelation__f_agg_""var""__maxlag_40",0.040266
28,"FMY__fft_coefficient__attr_""abs""__coeff_13",0.036828
36,"FMY__change_quantiles__f_agg_""mean""__isabs_Tru...",0.035102
12,FMY__fourier_entropy__bins_100,0.032824
34,FMY__partial_autocorrelation__lag_2,0.032509
46,"FMY__agg_autocorrelation__f_agg_""median""__maxl...",0.031082
3,FMY__fourier_entropy__bins_10,0.031058


In [87]:
feature_importance.to_excel('rf_ftest_features.xlsx')